In [1]:
import os, sys
import importlib
import torch

import numpy as np

from utils import load_torch, load

In [2]:
importlib.reload(load_torch)

data_folder = "quickdraw/train_simplified/"
access_file_generator = map(lambda x : data_folder + x + '.csv', load.classes)
trainGenerator = load_torch.ImageLoader(load.classes_1+load.classes_2+load.classes_3+load.classes_4, data_folder)

In [3]:
gen = trainGenerator.__iter__()

In [4]:
%matplotlib inline
X, y = next(gen)
print('X.shape:', X.shape)
print('y.shape:', y.shape)

X.shape: torch.Size([16, 1, 32, 32])
y.shape: torch.Size([16])


# Building PyTorch CNN Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag
import torch.optim as optim


class VanillaDoodle(nn.Module):
    # Here we define our network structure
    name="doodle-vanilla"
    def __init__(self, cdim=8):
        super(VanillaDoodle, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3).double() 
        self.conv2 = nn.Conv2d(6,16, 3).double()
        self.conv3 = nn.Conv2d(16,32,3).double()
        self.conv4 = nn.Conv2d(32,64,3).double()
        self.fc1   = nn.Linear(128,120).double()
        self.fc2   = nn.Linear(120, 84).double() 
        self.fc3   = nn.Linear(84,cdim).double()
        
    # Here we define one forward pass through the network
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = x.view(-1, int(self.num_flat_features(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # Determine the number of features in a batch of tensors
    def num_flat_features(self, x): 
        size = x.size()
        return np.prod(size[1:])


#net = VanillaDoodle(cdim=len(load.classes_1))

In [6]:
# Code derived from: https://github.com/pytorch/vision/blob/master/torchvision/models/inception.py

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class InceptionModule(nn.Module):

    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 12, kernel_size=1)

        self.branch3x3_1 = BasicConv2d(in_channels, 24, kernel_size=1)
        self.branch3x3_2a = BasicConv2d(24, 24, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3_2b = BasicConv2d(24, 24, kernel_size=(3, 1), padding=(1, 0))

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(32, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3a = BasicConv2d(24, 24, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3dbl_3b = BasicConv2d(24, 24, kernel_size=(3, 1), padding=(1, 0))

        self.branch_pool = BasicConv2d(in_channels, 16, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)
    
class DoodleInception(nn.Module):
    # Here we define our network structure
    name="doodle-inception"
    def __init__(self, cdim=8):
        super(DoodleInception, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3).double() 
        self.mod1  = InceptionModule(6).double()
        self.conv2 = nn.Conv2d(124, 128, 3).double()
        self.fc1   = nn.Linear(512,128).double()
        self.fc2   = nn.Linear(128,cdim).double()
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.mod1(x)), (2, 2))  #?x124x7x7
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))  #?x128x2x2
        x = x.view(-1, int(self.num_flat_features(x)))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    # Determine the number of features in a batch of tensors
    def num_flat_features(self, x): 
        size = x.size()
        return np.prod(size[1:])

In [7]:
B     = 128              # Minibatch size
T     = 10               # Number of epochs
gamma = .001             # learning rate
rho   = .9               # momentum

classes = load.classes_1+load.classes_2+load.classes_3+load.classes_4
print(len(classes))
    
net = DoodleInception(cdim=len(classes))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    net.parameters(),
    lr=gamma,
    momentum=rho
)

print(net)

32
DoodleInception(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (mod1): InceptionModule(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(6, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(12, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_1): BasicConv2d(
      (conv): Conv2d(6, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_2a): BasicConv2d(
      (conv): Conv2d(24, 24, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_2b): BasicConv2d(
      (conv): Conv2d(24, 24, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3dbl_1): BasicConv2d(
     

In [7]:
# use this code when starting training after connection loss
import pickle

with open("Inception CNN/32 classes/inception_cnn_32_training_acc", 'rb') as f:
    trainingAcc = pickle.load(f)
with open("Inception CNN/32 classes/inception_cnn_32_training_loss", 'rb') as f:
    trainingLoss = pickle.load(f)
with open("Inception CNN/32 classes/inception_cnn_32_val_acc", 'rb') as f:
    testingAcc = pickle.load(f)
with open("Inception CNN/32 classes/inception_cnn_32_val_loss", 'rb') as f:
    testingLoss = pickle.load(f)
    
with open("Inception CNN/32 classes/current_epoch", 'rb') as f:
    current_epoch = pickle.load(f)
print("current epoch: ",current_epoch)
with open("Inception CNN/32 classes/current_step", 'rb') as f:
    current_step = pickle.load(f)
print("current step: ",current_step)

classes = load.classes_1+load.classes_2+load.classes_3+load.classes_4
net = DoodleInception(cdim=len(classes))
net.load_state_dict(torch.load("Inception CNN/32 classes/inception_cnn_32.pt"))
net.eval()
print(net)

B     = 128              # Minibatch size
T     = 10     
gamma = .001             # learning rate
rho   = .9               # momentum
   
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    net.parameters(),
    lr=gamma,
    momentum=rho
)

current epoch:  7
current step:  15799
DoodleInception(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (mod1): InceptionModule(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(6, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(12, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_1): BasicConv2d(
      (conv): Conv2d(6, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_2a): BasicConv2d(
      (conv): Conv2d(24, 24, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch3x3_2b): BasicConv2d(
      (conv): Conv2d(24, 24, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (bn): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    

In [8]:
import pickle
if torch.cuda.is_available():
    net = net.cuda()

trainingAcc = [0]
testingAcc  = [0]
trainingLoss = []
testingLoss = []

for epoch in range(10):            
    gen = load_torch.ImageLoader(classes=load.classes_1+load.classes_2+load.classes_3+load.classes_4,root_location=data_folder,read_size=32,batch_size=150)
    for i, (images, labels) in enumerate(gen):
        
        vsplit = int(images.shape[0]*0.8)
        
        xtrain = images[:vsplit].cuda()
        ltrain = labels[:vsplit].cuda()
        
        xtest  = images[vsplit:].cuda()
        ltest  = labels[vsplit:].cuda()
        
        # Forward pass
        ypred = net(xtrain)
        loss = criterion(ypred, ltrain)
        train_acc  = 100 * np.mean(ltrain.data.cpu().numpy() == ypred.cpu().data.numpy ().T.argmax(axis =0))
        train_loss = loss.item()
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ypred = net(xtest)
        loss = criterion(ypred, ltest)
        test_acc  = 100 * np.mean(ltest.data.cpu().numpy() == ypred.cpu().data.numpy ().T.argmax(axis =0))
        test_loss = loss.item()
        
        del images
        del labels
        del ypred

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss: {:.4f} (val:{:.4f}), Accuracy:{:.2f}% (val:{:.2f}%)' .format(epoch+1, T, i+1, train_loss, test_loss, train_acc, test_acc))
            #model.save_state_dict('mytraining.pt')
     
            trainingAcc.append((0.1*train_acc) + (0.9*trainingAcc[-1]))
            trainingLoss.append(train_loss)
            testingAcc.append((0.1*test_acc) + (0.9*testingAcc[-1]))
            testingLoss.append(test_loss)
            
            #saving parameters/accuracies/losses
            torch.save(net.state_dict(), "Inception CNN/32 classes/inception_cnn_32.pt")
            with open("Inception CNN/32 classes/inception_cnn_32_training_acc", 'wb') as f:
                pickle.dump(trainingAcc, f)
            with open("Inception CNN/32 classes/inception_cnn_32_val_acc", 'wb') as f:
                pickle.dump(testingAcc, f)
            with open("Inception CNN/32 classes/inception_cnn_32_training_loss", 'wb') as f:
                pickle.dump(trainingLoss, f)
            with open("Inception CNN/32 classes/inception_cnn_32_val_loss", 'wb') as f:
                pickle.dump(testingLoss, f)
            with open("Inception CNN/32 classes/current_epoch", 'wb') as f:
                pickle.dump(epoch, f)
            with open("Inception CNN/32 classes/current_step", 'wb') as f:
                pickle.dump(i, f)



Epoch [8/10], Step [100], Loss: 0.7237 (val:0.5330), Accuracy:78.33% (val:86.67%)
Epoch [8/10], Step [200], Loss: 0.5782 (val:0.6182), Accuracy:85.00% (val:83.33%)
Epoch [8/10], Step [300], Loss: 0.5237 (val:0.4500), Accuracy:84.17% (val:86.67%)
Epoch [8/10], Step [400], Loss: 0.7198 (val:0.4666), Accuracy:77.50% (val:83.33%)
Epoch [8/10], Step [500], Loss: 0.6553 (val:0.7166), Accuracy:85.00% (val:86.67%)
Epoch [8/10], Step [600], Loss: 0.7716 (val:0.5298), Accuracy:82.50% (val:83.33%)
Epoch [8/10], Step [700], Loss: 0.3799 (val:0.5540), Accuracy:90.91% (val:88.00%)
Epoch [8/10], Step [800], Loss: 0.5400 (val:0.7263), Accuracy:83.33% (val:80.00%)
Epoch [8/10], Step [900], Loss: 0.4407 (val:0.7028), Accuracy:85.83% (val:80.00%)
Epoch [8/10], Step [1000], Loss: 0.5869 (val:0.1771), Accuracy:81.67% (val:93.33%)
Epoch [8/10], Step [1100], Loss: 0.5874 (val:0.6388), Accuracy:87.50% (val:80.00%)
Epoch [8/10], Step [1200], Loss: 0.5200 (val:0.5608), Accuracy:88.33% (val:90.00%)
Epoch [8/10],

Epoch [8/10], Step [10000], Loss: 0.5618 (val:0.8006), Accuracy:88.33% (val:86.67%)
Epoch [8/10], Step [10100], Loss: 0.6025 (val:0.6384), Accuracy:78.33% (val:83.33%)
Epoch [8/10], Step [10200], Loss: 0.8027 (val:0.7720), Accuracy:80.00% (val:80.00%)
Epoch [8/10], Step [10300], Loss: 0.6569 (val:0.5504), Accuracy:86.67% (val:86.67%)
Epoch [8/10], Step [10400], Loss: 0.7000 (val:1.0486), Accuracy:78.33% (val:76.67%)
Epoch [8/10], Step [10500], Loss: 0.7943 (val:0.6517), Accuracy:79.80% (val:84.00%)
Epoch [8/10], Step [10600], Loss: 0.5948 (val:1.0273), Accuracy:82.50% (val:73.33%)
Epoch [8/10], Step [10700], Loss: 0.6799 (val:0.3741), Accuracy:80.83% (val:83.33%)
Epoch [8/10], Step [10800], Loss: 0.6706 (val:0.5278), Accuracy:82.50% (val:76.67%)
Epoch [8/10], Step [10900], Loss: 0.5169 (val:1.5160), Accuracy:84.17% (val:70.00%)
Epoch [8/10], Step [11000], Loss: 0.6431 (val:0.6732), Accuracy:84.17% (val:83.33%)
Epoch [8/10], Step [11100], Loss: 0.5176 (val:0.7170), Accuracy:83.33% (val:

Epoch [8/10], Step [19800], Loss: 0.5849 (val:0.5834), Accuracy:87.50% (val:83.33%)
Epoch [8/10], Step [19900], Loss: 0.5310 (val:0.7701), Accuracy:84.17% (val:80.00%)
Epoch [8/10], Step [20000], Loss: 0.6412 (val:0.2697), Accuracy:81.67% (val:96.67%)
Epoch [8/10], Step [20100], Loss: 0.7411 (val:0.5220), Accuracy:70.83% (val:86.67%)
Epoch [8/10], Step [20200], Loss: 0.4695 (val:0.9279), Accuracy:84.17% (val:80.00%)
Epoch [8/10], Step [20300], Loss: 0.6115 (val:0.4987), Accuracy:80.81% (val:84.00%)
Epoch [8/10], Step [20400], Loss: 0.3987 (val:0.9854), Accuracy:84.17% (val:66.67%)
Epoch [8/10], Step [20500], Loss: 0.6480 (val:0.6806), Accuracy:80.83% (val:70.00%)
Epoch [8/10], Step [20600], Loss: 0.4807 (val:0.7320), Accuracy:88.33% (val:80.00%)
Epoch [8/10], Step [20700], Loss: 0.5254 (val:0.3642), Accuracy:86.67% (val:86.67%)
Epoch [8/10], Step [20800], Loss: 0.7985 (val:0.6225), Accuracy:78.33% (val:76.67%)
Epoch [8/10], Step [20900], Loss: 0.4825 (val:0.2673), Accuracy:86.67% (val:

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: generator 'ImageLoader.__iter__' raised StopIteration
  if sys.path[0] == '':


Epoch [9/10], Step [100], Loss: 0.6513 (val:0.4120), Accuracy:80.83% (val:86.67%)
Epoch [9/10], Step [200], Loss: 0.6834 (val:0.8513), Accuracy:77.50% (val:83.33%)
Epoch [9/10], Step [300], Loss: 0.5311 (val:0.7644), Accuracy:85.83% (val:76.67%)
Epoch [9/10], Step [400], Loss: 0.8144 (val:0.6899), Accuracy:75.00% (val:76.67%)
Epoch [9/10], Step [500], Loss: 0.4809 (val:0.4606), Accuracy:88.33% (val:83.33%)
Epoch [9/10], Step [600], Loss: 0.4829 (val:0.2658), Accuracy:88.33% (val:96.67%)
Epoch [9/10], Step [700], Loss: 0.6714 (val:0.5102), Accuracy:82.83% (val:84.00%)
Epoch [9/10], Step [800], Loss: 0.7654 (val:0.8037), Accuracy:80.83% (val:80.00%)
Epoch [9/10], Step [900], Loss: 0.7304 (val:0.7246), Accuracy:77.50% (val:76.67%)
Epoch [9/10], Step [1000], Loss: 0.5343 (val:0.5367), Accuracy:81.67% (val:83.33%)
Epoch [9/10], Step [1100], Loss: 0.6019 (val:1.0107), Accuracy:85.83% (val:66.67%)
Epoch [9/10], Step [1200], Loss: 0.5153 (val:0.4977), Accuracy:85.83% (val:86.67%)
Epoch [9/10],

Epoch [9/10], Step [10000], Loss: 0.5365 (val:0.5081), Accuracy:85.00% (val:90.00%)
Epoch [9/10], Step [10100], Loss: 0.6240 (val:0.9193), Accuracy:82.50% (val:80.00%)
Epoch [9/10], Step [10200], Loss: 0.7602 (val:0.8940), Accuracy:80.83% (val:76.67%)
Epoch [9/10], Step [10300], Loss: 0.5421 (val:1.0450), Accuracy:86.67% (val:73.33%)
Epoch [9/10], Step [10400], Loss: 0.7520 (val:0.5624), Accuracy:78.33% (val:80.00%)
Epoch [9/10], Step [10500], Loss: 0.8674 (val:0.8053), Accuracy:81.82% (val:84.00%)
Epoch [9/10], Step [10600], Loss: 0.6623 (val:0.6038), Accuracy:85.00% (val:86.67%)
Epoch [9/10], Step [10700], Loss: 0.6725 (val:0.3862), Accuracy:79.17% (val:86.67%)
Epoch [9/10], Step [10800], Loss: 0.5740 (val:0.5622), Accuracy:84.17% (val:80.00%)
Epoch [9/10], Step [10900], Loss: 0.4752 (val:0.5547), Accuracy:85.00% (val:80.00%)
Epoch [9/10], Step [11000], Loss: 0.5756 (val:0.7364), Accuracy:85.83% (val:83.33%)
Epoch [9/10], Step [11100], Loss: 0.5093 (val:0.3091), Accuracy:85.00% (val:

Epoch [9/10], Step [19800], Loss: 0.6637 (val:0.7079), Accuracy:80.83% (val:76.67%)
Epoch [9/10], Step [19900], Loss: 0.5706 (val:0.3476), Accuracy:84.17% (val:86.67%)
Epoch [9/10], Step [20000], Loss: 0.4901 (val:0.3818), Accuracy:85.00% (val:93.33%)
Epoch [9/10], Step [20100], Loss: 0.5337 (val:0.5178), Accuracy:85.00% (val:90.00%)
Epoch [9/10], Step [20200], Loss: 0.4874 (val:0.5620), Accuracy:81.67% (val:86.67%)
Epoch [9/10], Step [20300], Loss: 0.6026 (val:0.1376), Accuracy:84.85% (val:100.00%)
Epoch [9/10], Step [20400], Loss: 0.5319 (val:0.5077), Accuracy:83.33% (val:83.33%)
Epoch [9/10], Step [20500], Loss: 0.7254 (val:0.7736), Accuracy:82.50% (val:73.33%)
Epoch [9/10], Step [20600], Loss: 0.4325 (val:0.6886), Accuracy:88.33% (val:76.67%)
Epoch [9/10], Step [20700], Loss: 0.7302 (val:0.4967), Accuracy:82.50% (val:86.67%)
Epoch [9/10], Step [20800], Loss: 0.6175 (val:0.4715), Accuracy:82.50% (val:83.33%)
Epoch [9/10], Step [20900], Loss: 0.6576 (val:0.6592), Accuracy:85.00% (val

Epoch [10/10], Step [3700], Loss: 0.4884 (val:0.7345), Accuracy:87.50% (val:80.00%)
Epoch [10/10], Step [3800], Loss: 0.6277 (val:0.6184), Accuracy:84.17% (val:80.00%)
Epoch [10/10], Step [3900], Loss: 0.5720 (val:0.5332), Accuracy:85.83% (val:90.00%)
Epoch [10/10], Step [4000], Loss: 0.6923 (val:0.4480), Accuracy:79.17% (val:86.67%)
Epoch [10/10], Step [4100], Loss: 0.3452 (val:0.6180), Accuracy:88.33% (val:80.00%)
Epoch [10/10], Step [4200], Loss: 0.5539 (val:1.3548), Accuracy:84.85% (val:68.00%)
Epoch [10/10], Step [4300], Loss: 0.6302 (val:0.5206), Accuracy:82.50% (val:86.67%)
Epoch [10/10], Step [4400], Loss: 0.4533 (val:0.8822), Accuracy:85.83% (val:73.33%)
Epoch [10/10], Step [4500], Loss: 0.4326 (val:0.4237), Accuracy:90.83% (val:86.67%)
Epoch [10/10], Step [4600], Loss: 0.7257 (val:0.9683), Accuracy:80.83% (val:76.67%)
Epoch [10/10], Step [4700], Loss: 0.6163 (val:0.4288), Accuracy:83.33% (val:86.67%)
Epoch [10/10], Step [4800], Loss: 0.6100 (val:0.6157), Accuracy:83.33% (val:

Epoch [10/10], Step [13500], Loss: 0.8623 (val:0.3739), Accuracy:79.17% (val:86.67%)
Epoch [10/10], Step [13600], Loss: 0.6775 (val:0.5078), Accuracy:79.17% (val:86.67%)
Epoch [10/10], Step [13700], Loss: 0.6929 (val:0.4180), Accuracy:82.50% (val:86.67%)
Epoch [10/10], Step [13800], Loss: 0.3900 (val:0.4926), Accuracy:90.83% (val:86.67%)
Epoch [10/10], Step [13900], Loss: 0.8774 (val:0.8802), Accuracy:78.33% (val:73.33%)
Epoch [10/10], Step [14000], Loss: 0.8299 (val:0.1802), Accuracy:75.76% (val:96.00%)
Epoch [10/10], Step [14100], Loss: 0.5358 (val:0.3963), Accuracy:83.33% (val:90.00%)
Epoch [10/10], Step [14200], Loss: 0.5583 (val:0.9492), Accuracy:84.17% (val:80.00%)
Epoch [10/10], Step [14300], Loss: 0.5945 (val:0.7459), Accuracy:83.33% (val:80.00%)
Epoch [10/10], Step [14400], Loss: 0.6589 (val:0.6005), Accuracy:80.83% (val:83.33%)
Epoch [10/10], Step [14500], Loss: 0.6837 (val:0.4394), Accuracy:82.50% (val:86.67%)
Epoch [10/10], Step [14600], Loss: 0.6358 (val:0.4163), Accuracy:

Epoch [10/10], Step [23200], Loss: 0.4681 (val:0.3363), Accuracy:83.33% (val:90.00%)
Epoch [10/10], Step [23300], Loss: 0.6428 (val:0.9159), Accuracy:81.67% (val:80.00%)
Epoch [10/10], Step [23400], Loss: 0.8698 (val:0.7536), Accuracy:77.50% (val:83.33%)
Epoch [10/10], Step [23500], Loss: 0.5386 (val:0.5438), Accuracy:84.17% (val:83.33%)
Epoch [10/10], Step [23600], Loss: 0.4313 (val:1.1750), Accuracy:87.50% (val:66.67%)
Epoch [10/10], Step [23700], Loss: 0.4358 (val:0.1575), Accuracy:89.17% (val:100.00%)
Epoch [10/10], Step [23800], Loss: 0.6520 (val:0.7606), Accuracy:83.84% (val:84.00%)
Epoch [10/10], Step [23900], Loss: 0.7529 (val:0.3911), Accuracy:77.50% (val:90.00%)
Epoch [10/10], Step [24000], Loss: 0.5793 (val:0.5764), Accuracy:85.00% (val:80.00%)
Epoch [10/10], Step [24100], Loss: 0.6044 (val:0.2576), Accuracy:83.33% (val:90.00%)
Epoch [10/10], Step [24200], Loss: 0.5960 (val:1.2326), Accuracy:82.50% (val:73.33%)
Epoch [10/10], Step [24300], Loss: 0.4689 (val:0.4333), Accuracy